## Figure S2A 재현하기
*모든 코드는 Google Colab에서 실행한 코드를 그대로 붙여넣었습니다.


# 1. unique sequences 중복 및 error 확인하기

Mirlet7d 영역을 선택



In [ ]:
!grep -i Mirlet7d ../1/binfo1-datapack1/gencode.gtf

좌표 솎아내기. 이 파일을 바로 첫번째로 Genome Browser에 업로드해서 확인해보았습니다.

In [ ]:
!samtools view -b -o CLIP-let7d.bam ../1/binfo1-datapack1/CLIP-35L33G.bam chr13:48689488-48689590
!samtools view CLIP-let7d.bam

여기서부터는 시행착오를 겪은 기록이 섞여있습니다.

- pileup 파일로 시도해보기

In [ ]:
import pandas as pd

# 데이터 읽어오기
data = pd.read_csv('CLIP-let7d-gene.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])

# basereads 열에서 ^와 바로 뒤에 오는 문자 사이의 문자열 추출하기
data['errors'] = data['basereads'].str.extract(r'\^(.)', expand=False)

# errors 열에서 unique한 sequence와 그 개수 세기
error_counts = data.groupby('errors').size().reset_index(name='count')

# count 열이 7 이상인 행만 선택
frequent_errors = error_counts[error_counts['count'] >= 7]

# 결과 출력
print("7번 이상 나타나는 unique sequence들의 개수:", len(frequent_errors))
print("\n7번 이상 나타나는 unique sequence와 그 개수:")
print(frequent_errors)


- 중복 서열과 중복 수 출력해보기

In [ ]:
from collections import defaultdict

# SAM 파일 읽기
with open('CLIP-let7d-region-with-header.sam', 'r') as f:
    lines = f.readlines()

# 각 read의 서열 추출 및 중복 개수 세기
seq_counts = defaultdict(int)
for line in lines:
    if not line.startswith('@'):  # 헤더 라인 제외
        fields = line.strip().split('\t')
        seq = fields[9]  # 서열 정보가 있는 필드
        seq_counts[seq] += 1

# 5번 이상 중복된 서열만 선택
frequent_seqs = {seq: count for seq, count in seq_counts.items() if count >= 5}

# 결과 출력
for seq, count in frequent_seqs.items():
    print(f"{count}\t{seq}")

- BED 파일로 저장하기 

In [ ]:
from collections import defaultdict

# SAM 파일 읽기
with open('CLIP-let7d-region.sam', 'r') as f:
    lines = f.readlines()

# 각 read의 위치, 서열, error 정보 추출 및 중복 개수 세기
seq_counts = defaultdict(lambda: defaultdict(int))
for line in lines:
    if not line.startswith('@'):  # 헤더 라인 제외
        fields = line.strip().split('\t')
        pos = int(fields[3])  # 위치 정보
        seq = fields[9]  # 서열 정보
        error = fields[5]  # error 정보 (CIGAR string)
        seq_counts[seq][(pos, error)] += 1

# 7번 이상 중복된 서열만 선택
frequent_seqs = {seq: info for seq, info in seq_counts.items() if sum(info.values()) >= 7}

# BED 형식으로 출력
for seq, info in frequent_seqs.items():
    for (pos, error), count in info.items():
        start = pos - 1  # BED 형식은 0-based 좌표 사용
        end = pos + len(seq) - 1
        print(f"chr13\t{start}\t{end}\t{seq}\t{count}\t-\t{start}\t{end}\t0,0,255\t1\t{len(seq)}\t0\t{error}")

# BED 파일로 저장
with open('output.bed', 'w') as bed_file:
    for seq, info in frequent_seqs.items():
        for (pos, error), count in info.items():
            start = pos - 1  # BED 형식은 0-based 좌표 사용
            end = pos + len(seq) - 1
            bed_file.write(f"chr13\t{start}\t{end}\t{seq}\t{count}\t+\t{start}\t{end}\t0,0,255\t1\t{len(seq)}\t0\t{error}\n")

- BAM 파일 수정해서 저장하기

In [ ]:
# Unique sequence와 중복 횟수 추출
!samtools view CLIP-let7d.bam | cut -f 10 | sort | uniq -c > unique_sequences.txt

# 중복 횟수를 read name에 추가하여 새로운 BAM 파일 생성
!samtools view -H CLIP-let7d.bam > header.sam
!while read count seq; do \
    samtools view CLIP-let7d.bam | \
    awk -v count="$count" -v seq="$seq" 'BEGIN {{OFS="\t"}} $10 == seq {{$1 = count"_"$1; print}}' >> reads.sam; \
done < unique_sequences.txt
!cat header.sam reads.sam | samtools view -b > output1.bam

# 중간 파일 제거
!rm header.sam reads.sam

# BAM 파일 인덱싱
!samtools index output1.bam

- pysam 사용해보기

In [ ]:
import pysam

def add_duplicate_counts_to_bam(input_bam, output_bam):
    # Input BAM 파일 읽기
    bam_in = pysam.AlignmentFile(input_bam, "rb")

    # Output BAM 파일 쓰기 모드로 열기
    bam_out = pysam.AlignmentFile(output_bam, "wb", template=bam_in)

    # Unique sequence와 중복 횟수 저장할 딕셔너리
    unique_sequences = {}

    # Input BAM 파일에서 alignment 읽기
    for read in bam_in:
        seq = read.query_sequence
        if seq not in unique_sequences:
            unique_sequences[seq] = 1
        else:
            unique_sequences[seq] += 1

    # 중복 횟수를 read name에 추가하여 output BAM 파일에 쓰기
    for read in bam_in:
        seq = read.query_sequence
        count = unique_sequences[seq]
        read.query_name = f"{count}_{read.query_name}"
        bam_out.write(read)

    bam_in.close()
    bam_out.close()

# 함수 호출
add_duplicate_counts_to_bam("CLIP-let7d.bam", "pysam.bam")

# 2. Shannon's Entropy 구하기

In [ ]:
#대소문자 변환
def filter_bases(matches):
    return ''.join(base for base in matches.upper() if base in 'ACGT')

pileup['matches'] = pileup['matches'].apply(filter_bases)

def count_bases(matches):
    base_counts = {'A': 0, 'C': 0, 'G': 0, 'T': 0}

    # matches가 비어있는 경우 처리
    if matches.empty:
        return pd.Series(base_counts)

    for match in matches:
        for base in match:
            base_counts[base] += 1
    return pd.Series(base_counts)

#base 수 세기
base_counts = pileup.groupby('pos')['matches'].apply(count_bases).unstack().fillna(0)
base_counts.head(10)

def filter_bases(matches):
    return ''.join(base for base in matches if base in 'acgt')

pileup['matches'] = pileup['matches'].apply(filter_bases)

In [ ]:
import numpy as np

# Shannon entropy 계산 함수
def shannon_entropy(base_counts):
    total_counts = base_counts.sum()
    probs = base_counts / total_counts
    return -np.sum(probs * np.log2(probs))

# 각 position별로 Shannon entropy 계산
entropies = base_counts.apply(shannon_entropy, axis=1)

In [ ]:
import pandas as pd

entropies.to_csv('entropies.csv', header=True, index=True)
entropiess = pd.read_csv('entropies.csv', sep=',', names = ['pos', 'ent'])
entropiess.head(10)

In [ ]:
bedgraph = pd.DataFrame({
    'chrom': 'chr13',
    'start': pileup['pos'] - 1,
    'end': pileup['pos'],
    'entropy': entropiess['ent']
})

bedgraph = bedgraph.iloc[:-1]

bedgraph['start'] = bedgraph['start'].astype(int)
bedgraph['end'] = bedgraph['end'].astype(int)

with open('7d_entropy.bedgraph', 'w') as f:
    f.write('track type=bedGraph graphType=bar\n')
    bedgraph.to_csv(f, sep='\t', index=False, header=False, mode='a')

- matplotlib 으로 그려보기

In [ ]:
import matplotlib.pyplot as plt
positions = bedgraph['start']  # 각 position을 리스트로
entropies = bedgraph['entropy']
plt.figure(figsize=(10, 2))  # 그래프 크기 설정
plt.bar(positions, entropies, width=1)  # 막대 그래프 그리기
plt.xlim(min(positions), max(positions))  # x축 범위 설정
plt.ylim(0, max(entropies) * 1.1)  # y축 범위 설정
plt.xlabel('Position')
plt.ylabel("Shannon's entropy")
plt.title("Cross-linked peptide residue induced error rate")
plt.tight_layout()
plt.show()  # 그래프 표시